In [ ]:
import pandas as pd
from tqdm.std import tqdm
import json

## load data

In [ ]:
file_name = 'eye_new_annotation_v2_bm25+rank.csv' ## change it to your own path

In [ ]:
df = pd.read_csv(file_name)

In [ ]:
df

In [ ]:
## extract valid colums
df = df[["Element","GoldName","GoldID","standardConcept","conceptId"]]

In [ ]:
df.fillna("null",inplace=True)

In [ ]:
df

In [ ]:
## extracting goldId and predId
def extract_GoldIdAndPredId(df):
    goldIds = []
    predIds = []
    for row in df.itertuples():
        goldId = row.GoldID.split("||")
        goldIds.append(goldId)
        
        predId = row.conceptId
        predIds.append([predId])
    return goldIds, predIds
        

In [ ]:
goldIds, predIds = extract_GoldIdAndPredId(df)

In [ ]:
def calculate_acc_at_k(goldId, predId, k):
    correct = 0
    for gold, pred in zip(goldId, predId):
        if any(p in gold for p in pred[:k]):
            correct += 1
    return correct / len(goldId)


def calculate_precision_recall_f1(goldId, predId):
    true_positive = 0
    total_predicted = 0
    total_gold = 0
    
    for gold, pred in zip(goldId, predId):
        true_positive += len(set(gold) & set(pred))
        total_predicted += len(pred)
        total_gold += len(gold)
    
    precision = true_positive / total_predicted if total_predicted > 0 else 0
    recall = true_positive / total_gold if total_gold > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1


def calculate_precision_recall_at_k(goldId, predId, k):
    true_positive = 0
    total_predicted = 0
    total_gold = 0
    
    for gold, pred in zip(goldId, predId):
        true_positive += len(set(gold) & set(pred[:k]))
        total_predicted += min(len(pred[:k]), k)
        total_gold += len(gold)
    
    precision = true_positive / total_predicted if total_predicted > 0 else 0
    recall = true_positive / total_gold if total_gold > 0 else 0
    
    return precision, recall

In [ ]:
total_acc = {}

p, r, f1 = calculate_precision_recall_f1(goldIds,predIds)
total_acc['precision'] = p
total_acc['recall'] = r
total_acc['f1'] = f1

total_acc

## remove Mapping not found

In [ ]:
df1 = df[df.GoldID != "Mapping_Not_Found"]

In [ ]:
goldIds, predIds = extract_GoldIdAndPredId(df1)

In [ ]:
new_total_acc = {}

p, r, f1 = calculate_precision_recall_f1(goldIds,predIds)
new_total_acc['precision'] = p
new_total_acc['recall'] = r
new_total_acc['f1'] = f1

new_total_acc